In [58]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service


from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from webdriver_manager.microsoft import EdgeChromiumDriverManager



from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select




from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException, StaleElementReferenceException, NoSuchElementException
import time
import traceback

In [59]:
import pandas as pd
from datetime import datetime


pos = 0
autentica = pd.read_excel('ls.xlsx')
login = str(autentica['login'][pos])

autentica['senha'] = autentica['senha'].astype(str)
autentica['senha'] = autentica['senha'].str.zfill(11)

senha = autentica['senha'][pos]

display(autentica)
name = str(autentica['ubs'][pos])
data_e_hora_atuais = datetime.now()
data_e_hora = data_e_hora_atuais.strftime('%Y-%m-%d_%H-%M')

raceList = ['BRANCA', 'PRETA', 'AMARELA', 'PARDA', 'INDÍGENA', 'SEM INFORMAÇÃO']


# nav = webdriver.Chrome(ChromeDriverManager().install())
# nav = webdriver.Edge(EdgeChromiumDriverManager().install())


,login,senha,ubs
0,joana,05216791418,borges


In [60]:
def openEpHealph():
    
#     nav = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
#     nav = webdriver.Chrome(ChromeDriverManager().install())
    global nav
    nav = webdriver.Firefox(executable_path=GeckoDriverManager().install())
    nav.get("https://afogadosdaingazeira.ephealth.com.br/portal/login")
    
    # inserindo usuário
    nav.find_element(By.ID, 'EMail').click()
    nav.find_element(By.ID, 'EMail').send_keys(login)

    # inserindo senha 
    nav.find_element(By.ID, 'Password').click()
    nav.find_element(By.ID, 'Password').send_keys(senha)

    nav.find_element(By.XPATH, '/html/body/form/button').click()

In [61]:
def people():
    
    openEpHealph()
    print(f'Extração iniciada as: {datetime.now()}')
    nav.get("https://afogadosdaingazeira.ephealth.com.br/portal/individuo")
    
    nav.find_element(By.XPATH, '//*[@id="img_hipertensaoFlag"]').click()
    time.sleep(1)
    """
    race = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/form/div[3]/div[5]/span/span[1]/span/ul/li/input')
    race.click()
    time.sleep(1)
    race.send_keys(raceList[race_item])
    race.send_keys(Keys.ENTER)"""

    time.sleep(1)
    
    unity = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/form/div[4]/div[1]/span/span[1]/span/span[2]')
    unity.click()
    time.sleep(1)
    nav.find_element(By.XPATH,'/html/body/span/span/span[1]/input').send_keys(Keys.ENTER)
    time.sleep(1)
    nav.find_element(By.ID, 'button').click()
    
    time.sleep(1)
    nav.find_element(By.ID, 'card-toggle').click()
    time.sleep(10)

In [62]:
def get_table():
    
    list_links = []
    people()
    
    print(f'Iniciando lista de links: {datetime.now()}')

    exit = False
    
    peoples = nav.find_element(By.CLASS_NAME, 'dataTables_scrollBody')
    link = peoples.find_elements(By.TAG_NAME, 'a') 
    
    
    
    while exit is False:
        try:
            for a in link:
                
                list_links.append(a.get_attribute('href'))
                
            page = nav.find_element(By.CSS_SELECTOR, f"#individuo-list-datatable_next > a:nth-child(1)")

            page.click()
            time.sleep(10)                
            
        except (ElementClickInterceptedException, NoSuchElementException):
            if ElementClickInterceptedException:    
                print(ElementClickInterceptedException)
                
            if NoSuchElementException: 
                print(NoSuchElementException)
            print('acabaram as paginas')
            break
        
        except StaleElementReferenceException:
            peoples = nav.find_element(By.CLASS_NAME, 'dataTables_scrollBody')
            link = peoples.find_elements(By.TAG_NAME, 'a') 

    
    list_links = list(dict.fromkeys(list_links))
    print(f'lista de links finalizada as: {datetime.now()}')
    
    return list_links    


In [63]:
def get_indivivual_info():
    
#   name sex b_date cpf cns has dm med_rec race m_name
    name, sex, b_date, cpf, cns, has, dm, med_rec, race, m_name = [], [], [], [], [], [], [], [], [], []
    
    individual = get_table()
    
    print(f'Iniciando individuos : {datetime.now()}')
    for i in individual:
        
        time.sleep(1)
        
        nav.get(i)
        
        i_name = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/h5').text
        name.append(i_name)
        
        i_sex = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[1]').text
        sex.append(i_sex)
        
        i_bdate = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[4]/td[2]').text
        b_date.append(i_bdate)
        
        i_cpf = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[2]/td[2]').text
        cpf.append(i_cpf)
        
        i_cns = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[2]').text
        cns.append(i_cns)
        
        i_has = "sim"
        has.append(i_has)
        
        i_dm = "não"
        dm.append(i_dm)
        
        i_medrec = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[1]/td[2]').text
        med_rec.append(i_medrec)
        
        i_race = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[5]/td[2]').text
        race.append(i_race.upper())
        
        i_mname = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[6]/td[2]').text
        m_name.append(i_mname)
        
        time.sleep(1)

    print(f'Finalizando individuos : {datetime.now()}')

    dict_individual = {"name": name, "sex":sex, "b_date":b_date, "cpf":cpf, "cns":cns, "has":has, "dm":dm, "med_rec":med_rec, "race":race, "m_name":m_name}
    df_individual = pd.DataFrame(data=dict_individual)    
    
    print(f'Extração finalizada as: {datetime.now()}')    
    return df_individual

indi = get_indivivual_info()

Extração iniciada as: 2024-01-22 18:51:15.280913
Iniciando lista de links: 2024-01-22 18:51:32.296801
<class 'selenium.common.exceptions.ElementClickInterceptedException'>
<class 'selenium.common.exceptions.NoSuchElementException'>
acabaram as paginas
lista de links finalizada as: 2024-01-22 18:53:30.182075
Iniciando individuos : 2024-01-22 18:53:30.182143
Finalizando individuos : 2024-01-22 19:29:38.596802
Extração finalizada as: 2024-01-22 19:29:38.600403


In [64]:
display(indi)

,name,sex,b_date,cpf,cns,has,dm,med_rec,race,m_name
0,ABNER DA SILVA VASCONCELLOS,"Masculino, 42 anos 9 meses e 25 dias",28/03/1981,03648041495,CNS: 700009126573002,sim,não,--,PARDA,GILDA MARIA DA SILVA VASCONCELOS
1,ACIDALIA AMORIM VIANA,"Feminino, 80 anos 0 mês e 0 dia",22/01/1944,58938320430,CNS: 705007474533958,sim,não,821,BRANCA,IZABEL MASCENA AMORIM
2,ADEILTON MARQUES DE BRITO,"Masculino, 58 anos 2 meses e 12 dias",10/11/1965,40301710449,CNS: 708608165060990,sim,não,1025,BRANCA,(desconhecido)
3,ADELMO FREITAS DE BRITO,"Masculino, 58 anos 9 meses e 19 dias",03/04/1965,ADELMO,CNS: 700600993771369,sim,não,203,BRANCA,MARIA DE LOURDES DE FREITAS
4,ADEMIR RODRIGUES DA SILVA,"Masculino, 87 anos 5 meses e 17 dias",05/08/1936,ADEMIR,CNS: 700106953790117,sim,não,01003,BRANCA,FRANCISCA SIMÕES DA SILVA
...,...,...,...,...,...,...,...,...,...,...
578,WALDIRENE FERREIRA DE SANTANA,"Feminino, 56 anos 4 meses e 21 dias",01/09/1967,VALDIRENE,CNS: 700803983690790,sim,não,--,PARDA,HELENA FERREIRA DE SOUZA
579,WALMICK GUIMARAES MARTINS,"Masculino, 81 anos 1 mês e 30 dias",23/11/1942,02187310434,CNS: 898002350120734,sim,não,--,PARDA,MARIA GUIMARAES MARTINS
580,ZÉLIA ROSA CARVALHO,"Feminino, 73 anos 5 meses e 0 dia",22/08/1950,86436511415,CNS: 700008847197400,sim,não,223,BRANCA,ROSA TERTULIANA DE JESUS
581,ZULMIRA DE SIQUEIRA CARDEAL,"Feminino, 76 anos 4 meses e 20 dias",02/09/1947,38666677449,CNS: 700103968121418,sim,não,--,BRANCA,SALVINA FRANCISCA DE SIQUEIRA
